In [1]:
from datetime import datetime
from decimal import Decimal
from math import floor
import numpy as np
import pandas as pd
from os.path import join

In [2]:
path = '/home/b3arjuden/crocket/sql_data/PRODUCTION40'

file = 'BTC-ETH.csv'

In [3]:
file_path = join(path, file)

data = pd.read_csv(file_path, 
                   dtype={'time': str, 'buy_order': int, 'sell_order': int},
                   converters={'price': Decimal,
                               'wprice': Decimal,
                               'base_volume': Decimal,
                               'buy_volume': Decimal,
                               'sell_volume': Decimal})

In [19]:
def time_transform(df, n):
    
    df1 = pd.DataFrame()
    
    nrows = n * floor(df.shape[0] / n)
    df = df.iloc[:nrows, :]
    
    df1['time'] = df.loc[df.index[::n], 'time'].reset_index(drop=True)
    df1['wprice'] = ((df.loc[:, 'wprice'] * df.loc[:, 'base_volume']).groupby(df.index // n).sum() / 
                     df.loc[:, 'base_volume'].groupby(df.index // n).sum()).apply(lambda x: x.quantize(Decimal(10) ** -8))
    df1['buy_volume'] = df.loc[:, 'buy_volume'].groupby(df.index // n).sum()
    df1['sell_volume'] = df.loc[:, 'sell_volume'].groupby(df.index // n).sum()
    df1['buy_order'] = df.loc[:, 'buy_order'].groupby(df.index // n).sum()
    df1['sell_order'] = df.loc[:, 'sell_order'].groupby(df.index // n).sum()
    
    return df1

In [20]:
df = time_transform(data, 3)